## Background
The aim of this research is to use AI-assisted technology to automatically generate interaction claims from additional, previously under utilized sources of data. In doing so, we can increase data parity between the published state of knowledge and what is available in databases.  
  
**Approach**:  
  
1. Take an existing pretrained model and use train it to perform **entity recognition** of drugs, genes, variants, and phenotypes. 
2. Either (1): identify a suitable model or, (2): train an existing pretrained model further to perform **text summarization** to summarize a label to link drugs, genes, variants, phenotypes within one individual 'claim'. (**relationship classification**??)

## Load Labels using Datasets
Dataset is obtained from OpenFDA json resource. Pre-sectioned labels were imported using the json library and converted to a pandas dataframe. 

In [38]:
from datasets import Dataset
import pandas as pd

drugs_at_fda = pd.read_excel('../data/openfda.xlsx').reset_index(drop=True).drop('Unnamed: 0', axis=1)

drugs_at_fda


,brand_name,adverse_reactions,indications_and_usage,contraindications,warnings_and_cautions,warnings,precautions,pharmacokinetics,purpose,clinical_pharmacology,active_ingredient,stop_use,boxed_warning,pharmacodynamics,pharmacogenomics
0,AMOXICILLIN AND CLAVULANATE POTASSIUM,ADVERSE REACTIONS SECTION The following are di...,INDICATIONS & USAGE SECTION To reduce the deve...,CONTRAINDICATIONS SECTION Amoxicillinfor oral ...,WARNINGS AND PRECAUTIONS SECTION 5.1 Anaphylac...,NaN,NaN,NaN,NaN,CLINICAL PHARMACOLOGY SECTION 12.1 Mechanism o...,NaN,NaN,NaN,NaN,NaN
1,UNDA 312,NaN,Uses For the relief of symptoms associated wit...,NaN,NaN,Warnings Sore throat warning: Severe or persis...,NaN,NaN,Uses For the relief of symptoms associated wit...,NaN,Active ingredients Each drop contains: Angelic...,Stop use and ask a doctor if Cough persists fo...,NaN,NaN,NaN
2,SUN PROTECT LIP BALM SPF 30,NaN,Uses Helps prevent sunburn. If used as directe...,NaN,NaN,Warnings For external use only. Do not use on ...,NaN,NaN,Purpose Sunscreen,NaN,Drug Facts Active ingredients Non Nano Zinc Ox...,NaN,NaN,NaN,NaN
3,LOSARTAN POTASSIUM AND HYDROCHLOROTHIAZIDE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Potassium Phosphates,6 ADVERSE REACTIONS The following clinically s...,1 INDICATIONS AND USAGE Potassium Phosphates I...,4 CONTRAINDICATIONS Potassium Phosphates Injec...,5 WARNINGS AND PRECAUTIONS Serious Cardiac Adv...,NaN,NaN,12.3 Pharmacokinetics Distribution Approximate...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215910,Carbidopa and Levodopa,ADVERSE REACTIONS The most common adverse reac...,INDICATIONS AND USAGE Carbidopa and levodopa t...,CONTRAINDICATIONS Nonselective monoamine oxida...,NaN,WARNINGS When carbidopa and levodopa tablets a...,"PRECAUTIONS General As with levodopa, periodic...",Pharmacokinetics Carbidopa reduces the amount ...,NaN,CLINICAL PHARMACOLOGY Mechanism of Action Park...,NaN,NaN,NaN,Pharmacodynamics When levodopa is administered...,NaN
215911,REFRESH Optive Mega-3,NaN,"Uses For the temporary relief of burning, irri...",NaN,NaN,Warnings For external use only. To avoid conta...,NaN,NaN,Purpose Eye lubricant Eye lubricant Eye lubricant,NaN,Active ingredients Carboxymethylcellulose sodi...,Stop use and ask a doctor if you experience ey...,NaN,NaN,NaN
215912,Creon,6 ADVERSE REACTIONS The most serious adverse r...,1 INDICATIONS AND USAGE CREON ® is indicated f...,4 CONTRAINDICATIONS None. None ( 4 ),5 WARNINGS AND PRECAUTIONS Fibrosing colonopat...,NaN,NaN,12.3 Pharmacokinetics The pancreatic enzymes i...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
215913,Losartan Potassium and Hydrochlorothiazide,6 ADVERSE REACTIONS Most common adverse reacti...,1 INDICATIONS AND USAGE Losartan potassium and...,4 CONTRAINDICATIONS Losartan potassium and hyd...,5 WARNINGS AND PRECAUTIONS Hypotension: Correc...,NaN,NaN,12.3 Pharmacokinetics Losartan Potassium Absor...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,WARNING: FETAL TOXICITY When pregnancy is dete...,12.2 Pharmacodynamics Losartan Potassium Losar...,NaN


## Named Entity Recognition (NER) for Pharmacokinetics / Pharmacodynamics / Pharmacogenomics
The model being utilizes for tag generation for this section is the bioNLP System for BioNER and BioNEN: https://github.com/librairy/bio-ner, 

- Genetics: https://huggingface.co/alvaroalon2/biobert_genetic_ner  
- Diseases: https://huggingface.co/alvaroalon2/biobert_diseases_ner
- Chemicals: https://huggingface.co/alvaroalon2/biobert_chemical_ner
  
Example: https://user-images.githubusercontent.com/72864707/120455516-20a28f80-c395-11eb-97a8-fb54b017eaab.png
  
Biomedical Named Entity Recognition and Normalization of Diseases, Chemicals, and Genetic entity classes through the use of state-of-the-art models. The core piece in the modelling of the text entities recognition will be BioBERT. The model was a fine-tuned version of BioBERT for other instances of genetico, quimicos, and enfermedades as a part of a master's thesis from the Escuela Tecnica Superior (ETS) Universidad Politecnica de Madrid. Two additional corpi were used to train each entity class: Diseases (BC5CDR - Diseases, NCBI - Diseases), Chemicals (BC4CHEMD, BC5CDR - Chemicals), and Genes/Proteins (JNLPBA, BC2GM)
  
https://oa.upm.es/67933/1/TFM_ALVARO_ALONSO_CASERO.pdf 

### Main NER Pipeline
Given a header corresponding to a labels from Drugs@FDA, utilize a series of fine-tuned BioBERT transformer models to perform named entity recognition for all Chemical, Genomic, and Disease entities

In [43]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from transformers import AutoTokenizer
import pandas as pd

model_checkpoint = "alvaroalon2/biobert_genetic_ner"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
pipe_gene = pipeline("token-classification", model="alvaroalon2/biobert_genetic_ner",aggregation_strategy='first')
pipe_chemical = pipeline("token-classification", model="alvaroalon2/biobert_chemical_ner",aggregation_strategy='first')
pipe_disease = pipeline("token-classification", model="alvaroalon2/biobert_diseases_ner",aggregation_strategy='first')


def pre_process(label_header,df):
    def get_tokens(entry):
        return {'tokens': tokenizer(entry[label_header]).tokens() }

    f = pd.DataFrame(df, columns=[label_header])
    dataset = Dataset.from_pandas(f)

    print(f'UNFILTERED: {len(dataset[label_header])}')

    dataset = dataset.filter(lambda x: x[label_header] is not None)
    dataset = dataset.map(get_tokens)
    print(f'After NONE CHECK: {len(dataset[label_header])}')

    return(dataset)


def pipe(label_header,dataset):
    def generate_genomic_ner(entry):
        return {'genomic_ner': pipe_gene(entry[label_header]) }

    def generate_chemical_ner(entry):
        return {'chemical_ner': pipe_chemical(entry[label_header]) }

    def generate_diseases_ner(entry):
        return {'diseases_ner': pipe_disease(entry[label_header]) }

    def merged_ner_record(entry):
        return {'merged_ner_groups': entry['genomic_ner'] + entry['chemical_ner'] + entry['diseases_ner']}

    dataset = dataset.map(generate_genomic_ner)
    dataset = dataset.map(generate_chemical_ner)
    dataset = dataset.map(generate_diseases_ner)

    dataset = dataset.map(merged_ner_record)

    return(dataset)

def post_process(label_header,dataset):
    def initialize_ner_labels(entry):
        return {'token_labels': ['0'] * len(tokenizer(entry[label_header]).tokens())}

    def align_genomic_labels(entry):
        tags = ['0'] * len(tokenizer(entry[label_header]).tokens())

        for ner in entry['genomic_ner']: # TO DO, generalize for all headers
            tags[ner['index']] = str(ner['entity'])

        return {'genomic_labels': tags}

    def align_chemical_labels(entry):
        tags = ['0'] * len(tokenizer(entry[label_header]).tokens())

        for ner in entry['chemical_ner']: # TO DO, generalize for all headers
            tags[ner['index']] = str(ner['entity'])

        return {'chemical_labels': tags}

    def align_disease_labels(entry):
        tags = ['0'] * len(tokenizer(entry[label_header]).tokens())

        for ner in entry['diseases_ner']: # TO DO, generalize for all headers
            tags[ner['index']] = str(ner['entity'])

        return {'disease_labels': tags}

    dataset = dataset.map(align_genomic_labels)
    dataset = dataset.map(align_chemical_labels)
    dataset = dataset.map(align_disease_labels)


    return(dataset)

def build_dataframe(feature,dataset):

    # Build DF
    df = pd.DataFrame()

    for entry in dataset[feature]:
        tdf = pd.DataFrame(entry)
        df = pd.concat([df,tdf]).reset_index(drop=True)

    return(df)

In [47]:
dataset['chemical_ner']

[[{'end': 44,
   'entity_group': 'CHEMICAL',
   'score': 0.9999901652336121,
   'start': 37,
   'word': 'HMG - CoA'},
  {'end': 89,
   'entity_group': 'CHEMICAL',
   'score': 0.9999961853027344,
   'start': 77,
   'word': 'rosuvastatin'},
  {'end': 184,
   'entity_group': 'CHEMICAL',
   'score': 0.9999980926513672,
   'start': 172,
   'word': 'rosuvastatin'},
  {'end': 523,
   'entity_group': 'CHEMICAL',
   'score': 0.9999979734420776,
   'start': 511,
   'word': 'rosuvastatin'}],
 [{'end': 240,
   'entity_group': 'CHEMICAL',
   'score': 0.9999982118606567,
   'start': 230,
   'word': 'metoprolol'},
  {'end': 276,
   'entity_group': 'CHEMICAL',
   'score': 0.9999980926513672,
   'start': 266,
   'word': 'metoprolol'}],
 [{'end': 240,
   'entity_group': 'CHEMICAL',
   'score': 0.9999982118606567,
   'start': 230,
   'word': 'metoprolol'},
  {'end': 276,
   'entity_group': 'CHEMICAL',
   'score': 0.9999980926513672,
   'start': 266,
   'word': 'metoprolol'}],
 [{'end': 267,
   'entity_gr

In [49]:
dataset['genomic_ner']

[[{'end': 54,
   'entity_group': 'GENETIC',
   'score': 0.9999889731407166,
   'start': 37,
   'word': 'HMG - CoA reductase'},
  {'end': 107,
   'entity_group': 'GENETIC',
   'score': 0.999990701675415,
   'start': 100,
   'word': 'OATP1B1'},
  {'end': 129,
   'entity_group': 'GENETIC',
   'score': 0.5277884006500244,
   'start': 118,
   'word': 'transporter'},
  {'end': 318,
   'entity_group': 'GENETIC',
   'score': 0.9999926090240479,
   'start': 311,
   'word': 'OATP1B1'},
  {'end': 335,
   'entity_group': 'GENETIC',
   'score': 0.9975832104682922,
   'start': 321,
   'word': 'SLCO1B1 521T > C'},
  {'end': 391,
   'entity_group': 'GENETIC',
   'score': 0.9974077939987183,
   'start': 376,
   'word': 'SLCO1B1 521 C / C'}],
 [{'end': 28,
   'entity_group': 'GENETIC',
   'score': 0.9999911785125732,
   'start': 22,
   'word': 'CYP2D6'},
  {'end': 131,
   'entity_group': 'GENETIC',
   'score': 0.9999901056289673,
   'start': 125,
   'word': 'CYP2D6'},
  {'end': 194,
   'entity_group': '

In [50]:
dataset = pre_process('pharmacogenomics',drugs_at_fda)
dataset = pipe('pharmacogenomics',dataset)
# dataset = post_process('pharmacogenomics',dataset) TO DO, REFORMAT FOR ENTITY GROUPS :(
evaluated = build_dataframe('merged_ner_groups',dataset)



UNFILTERED: 215915


Filter:   0%|          | 0/215915 [00:00<?, ? examples/s]

Map:   0%|          | 0/355 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors


After NONE CHECK: 355


...

Map:   0%|          | 0/355 [00:00<?, ? examples/s]

Map:   0%|          | 0/355 [00:00<?, ? examples/s]

Map:   0%|          | 0/355 [00:00<?, ? examples/s]

Map:   0%|          | 0/355 [00:00<?, ? examples/s]

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/m6/b6y4g9114836jky8p81w12mchscvrj/T/ipykernel_95788/2999554994.py:3 in <cell │
│ line: 3>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/m6/b6y4g9114836jky8p81w12mchscvrj/T/ipykernel_95788/2999554994.py'          │
│ /var/folders/m6/b6y4g9114836jky8p81w12mchscvrj/T/ipykernel_95788/1805754152.py:78 in      │
│ post_process                                                                              │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/m6/b6y4g9114836jky8p81w12mchscvrj/T/ipykernel_95788/1805754152.py'          │
│                                                                                           │
│ /Users/mjc014/.pyenv/versions/3.10.4/lib/python3.10/site-packages/datasets/arrow_dataset. │
│ py:578 in wrapper                                                                         │
│                                                                                           │
│    575 │   │   else:                                                                      │
│    576 │   │   │   self: "Dataset" = kwargs.pop("self")                                   │
│    577 │   │   # apply actual function                                                    │
│ ❱  578 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)         │
│    579 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) el │
│    580 │   │   for dataset in datasets:                                                   │
│    581 │   │   │   # Remove task templates if a column mapping of the template is no long │
│                                                                                           │
│ /Users/mjc014/.pyenv/versions/3.10.4/lib/python3.10/site-packages/datasets/arrow_dataset. │
│ py:543 in wrapper                                                                         │
│                                                                                           │
│    540 │   │   │   "output_all_columns": self._output_all_columns,                        │
│    541 │   │   }                                                                          │
│    542 │   │   # apply actual function                                                    │
│ ❱  543 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)         │
│    544 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) el │
│    545 │   │   # re-apply format to the output                                            │
│    546 │   │   for dataset in datasets:                                                   │
│                                                                                           │
│ /Users/mjc014/.pyenv/versions/3.10.4/lib/python3.10/site-packages/datasets/arrow_dataset. │
│ py:3073 in map                                                                            │
│                                                                                           │
│   3070 │   │   │   │   │   leave=False,                                                   │
│   3071 │   │   │   │   │   desc=desc or "Map",                                            │
│   3072 │   │   │   │   ) as pbar:                                                         │
│ ❱ 3073 │   │   │   │   │   for rank, done, content in Dataset._map_single(**dataset_kwarg │
│   3074 │   │   │   │   │   │   if done:           

In [ ]:
evaluated['entity_group'].value_counts()

In [ ]:
evaluated[evaluated['entity_group']!='0'].to_excel('entity-results.xlsx')
evaluated[evaluated['entity_group']!='0']

## Normalization Pipeline
Utilize the VICC Therapy, Gene, and Disease normalizers to perform normalization of extracted entities TO DO: Variant normalizer can work with example [4]
 Compare vs Civic Mine, other examples (Manuela Benary Berlin, etc)

In [ ]:
import requests
from tqdm import tqdm

evaluated['match_type'] = None
evaluated['concept_id'] = None

therapy_norm_url = 'https://normalize.cancervariants.org/therapy/normalize?q='
disease_norm_url = 'https://normalize.cancervariants.org/disease/normalize?q='
gene_norm_url = 'https://normalize.cancervariants.org/gene/normalize?q='

index_pos = 0
for entry in tqdm(evaluated['word']):
    if evaluated['entity_group'][index_pos]=='CHEMICAL':
        r = requests.get(therapy_norm_url + entry)
        try:
            evaluated['match_type'][index_pos] = r.json()['match_type']
            evaluated['concept_id'][index_pos] = r.json()['therapy_descriptor']['therapy_id']
        except:
            pass

    elif evaluated['entity_group'][index_pos]=='DISEASE':
        r = requests.get(disease_norm_url + entry)
        try:
            evaluated['match_type'][index_pos] = r.json()['match_type']
            evaluated['concept_id'][index_pos] = r.json()['disease_descriptor']['disease_id']
        except:
            pass

    elif evaluated['entity_group'][index_pos]=='GENETIC':
        r = requests.get(gene_norm_url + entry)
        try:
            evaluated['match_type'][index_pos] = r.json()['match_type']
            evaluated['concept_id'][index_pos] = r.json()['gene_descriptor']['id']
        except:
            pass

    else:
        pass

    index_pos += 1

In [ ]:
stats = evaluated[evaluated['entity_group']=='DISEASE']
print('Normalized (DISEASE): ' + str(len(stats[stats['concept_id'].isnull()==False])) + ' / ' + str(len(stats['concept_id'])) + ' (' + str((len(stats[stats['concept_id'].isnull()==False])/len(stats['concept_id']))*100) + ')' )

stats = evaluated[evaluated['entity_group']=='CHEMICAL']
print('Normalized (CHEMICAL): ' + str(len(stats[stats['concept_id'].isnull()==False])) + ' / ' + str(len(stats['concept_id'])) + ' (' + str((len(stats[stats['concept_id'].isnull()==False])/len(stats['concept_id']))*100) + ')' )

stats = evaluated[evaluated['entity_group']=='GENETIC']
print('Normalized (GENETIC): ' + str(len(stats[stats['concept_id'].isnull()==False])) + ' / ' + str(len(stats['concept_id'])) + ' (' + str((len(stats[stats['concept_id'].isnull()==False])/len(stats['concept_id']))*100) + ')' )

In [ ]:
evaluated = evaluated[evaluated['entity_group']!="0"]
evaluated.sort_values(by='match_type').reset_index(drop=True)
# evaluated.to_csv('ner_norm_failure_20231017.csv',sep='\t')

In [ ]:
stats[stats['concept_id'].isnull()==True]['word'].value_counts()

## Graph Normalization Results

In [17]:
evaluated['normalized'] = evaluated['match_type']>0
evaluated_g = evaluated.groupby(['entity_group','normalized']).count()[['word']].unstack(level=1)['word']

evaluated_g

normalized,False,True
entity_group,,
CHEMICAL,659,966
DISEASE,116,115
GENETIC,1171,1129


In [18]:
evaluated_g['percent_true'] = (evaluated_g[True] / (evaluated_g[True] + evaluated_g[False])) * 100
evaluated_g['percent_false'] = (evaluated_g[False] / (evaluated_g[True] + evaluated_g[False])) * 100
evaluated_g = evaluated_g.sort_values(['percent_true'],ascending=False)
evaluated_g

normalized,False,True,percent_true,percent_false
entity_group,,,,
CHEMICAL,659,966,59.446154,40.553846
DISEASE,116,115,49.783550,50.216450
GENETIC,1171,1129,49.086957,50.913043


In [24]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace (
    go.Bar(
        x = evaluated_g.index,
        y = evaluated_g['percent_true'],
        name = 'Normalized',
        texttemplate="%{value:.2f}",
        marker=dict(color='#444444')
    )
)

fig.add_trace(
    go.Bar(
        x = evaluated_g.index,
        y = evaluated_g['percent_false'],
        name = 'Not Normalized',
        texttemplate="%{value:.2f}",
        marker=dict(color='#dddddd')

    )
)

fig.update_xaxes(linecolor='black',title='Label')
fig.update_yaxes(linecolor='black',title='% Normalized')

fig.update_layout(barmode='stack',plot_bgcolor='#FFF',title='Normalization of Extracted Terms from FDA Labels')

fig.show()
# fig.write_html('scratch/normalization_graph.html')


## Scratch

#### Rejoin Chunks

In [ ]:
def recombine_ner_chunks(entry):
    all_tags = []
    all_words = []
    for tag_set in entry['merged_ner_groups']:
        all_tags.append(tag_set['entity'])
        all_words.append(tag_set['word'])

    i = 0
    chunk_number = []
    for tag in all_tags:
        if tag.startswith('B-') == True:
            i += 1
            chunk_number.append(i)

        if tag.startswith('I-') == True:
            chunk_number.append(i)

    position = 0
    rebuilt = []
    word = ''
    for chunk_indicator in chunk_number:
        current_word = all_words[position]
        try:
            if chunk_indicator == chunk_number[position + 1]:
                # don't append
                word = word + ' ' + current_word # build word
                position += 1# iterate
            else:
                word = word + ' ' + current_word
                rebuilt.append(word.strip()) # append
                word = '' # start new word
                position += 1 # iterate
        except:
            word = word + ' ' + current_word
            rebuilt.append(word.strip())


    return({'ner_chunk_indicators': chunk_number, 'individual_ner_words' : all_words, 'ner_chunks': rebuilt })

In [ ]:
dataset = dataset.map(recombine_ner_chunks)
dataset

Map:   0%|          | 0/355 [00:00<?, ? examples/s]

Dataset({
    features: ['pharmacogenomics', 'genomic_ner', 'chemical_ner', 'diseases_ner', 'merged_ner_groups', 'ner_chunk_indicators', 'individual_ner_words', 'ner_chunks'],
    num_rows: 355
})

#### Convert free-text to JSON representations 
TO DO: This section is Likely no longer relevant, but keeping for now

**(9/14)**: In previous exercise with Kindred, PDF labels from Drugs@FDA were used as follows:  
  
1. PDFs were downloaded 
2. PDFs were converted to free-text files.  
3. Free-text files were then separated into different sections for Indications, Contraindications, and Adverse Effects.  (possible LLM task?)
    
This separation was chosen as the types of named entities and relationships would be largely identical, but with different inferred meanings dependent on the section they were present within (this seemed a hard problem to address). Additionally, due to the labels being highly unstructured and designed for visual, human understanding, the conversion from PDF to free-text could be extremely messy. 

**(9/28)**: In this exercise, Drugs@FDA labels were obtained from the OpenFDA resource and converted from JSON format to a pandas dataframe (excel file). Some of these fields are still messy, but they are now less messy than the previous method and are pre-sectioned.

I am thinking now about how to re-format this dataset for use in HuggingFace models. The aim of this research is to use AI-assisted technology to automatically generate interaction claims from additional, previously under utilized sources of data. In doing so, we can increase data parity between the published state of knowledge and what is available in databases.  
  
**Approach**:  
  
1. Take an existing pretrained model and use train it to perform **entity recognition** of drugs, genes, variants, and phenotypes. 
2. Either (1): identify a suitable model or, (2): train an existing pretrained model further to perform **text summarization** to summarize a label to link drugs, genes, variants, phenotypes within one individual 'claim'. (**relationship classification**??)

In thinking about this, I think the data should eventually be formatted like this:

In [ ]:
# JSON format for Adverse Effects section
{
    "meta": { "label": <identifier>,
                "drug": <drug label>,
                "type": <type of page, i.e. indication, adverse effects, contraindications>,
                "url": <url of label download>,
            },
    "adverse_effects": <free text dump>,
    "tokens": [...],
    "pos_tags": [...], # IDs
    "chunk_tags": [...], # IDs
    "ner_tags": [...], # IDs
    "id": <identifier for data point>

}

In [112]:
variable = 'awesome'

print(f'Anastasia is {variable}.')
print("Anastasia is " + variable + ".")

Anastasia is awesome.
Anastasia is awesome.
